# Настройка среды

In [ ]:
%%bash

apt-get update -qq && apt-get install -qq fluidsynth

pip install -qU note-seq pygtrie tensor2tensor pyfluidsynth bokeh==2.4.3
sed -i -e '46,49d' -e '64,67d' /usr/local/lib/python3.10/dist-packages/tensorflow_probability/python/internal/prefer_static.py
sed -i 's/np.bool/bool/g' /usr/local/lib/python3.10/dist-packages/tensorflow_probability/python/mcmc/sample_halton_sequence.py
sed -i 's/_kwargs = spec_.kwargs.copy()/_kwargs = spec_.kwargs.copy() if spec_.kwargs else {}/g' /usr/local/lib/python3.10/dist-packages/gym/envs/registration.py

git clone --branch=main https://github.com/magenta/magenta
mv magenta magenta_tmp; mv magenta_tmp/* .; rm -rf magenta_tmp
rm ./magenta/__init__.py
rm ./magenta/models/score2perf/__init__.py

Selecting previously unselected package libqt5core5a:amd64.
(Reading database ... 121918 files and directories currently installed.)
Preparing to unpack .../00-libqt5core5a_5.15.3+dfsg-2ubuntu0.2_amd64.deb ...
Unpacking libqt5core5a:amd64 (5.15.3+dfsg-2ubuntu0.2) ...
Selecting previously unselected package libevdev2:amd64.
Preparing to unpack .../01-libevdev2_1.12.1+dfsg-1_amd64.deb ...
Unpacking libevdev2:amd64 (1.12.1+dfsg-1) ...
Selecting previously unselected package libmtdev1:amd64.
Preparing to unpack .../02-libmtdev1_1.1.6-1build4_amd64.deb ...
Unpacking libmtdev1:amd64 (1.1.6-1build4) ...
Selecting previously unselected package libgudev-1.0-0:amd64.
Preparing to unpack .../03-libgudev-1.0-0_1%3a237-2build1_amd64.deb ...
Unpacking libgudev-1.0-0:amd64 (1:237-2build1) ...
Selecting previously unselected package libwacom-common.
Preparing to unpack .../04-libwacom-common_2.2.0-1_all.deb ...
Unpacking libwacom-common (2.2.0-1) ...
Selecting previously unselected package libwacom9:a

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
panel 1.3.8 requires bokeh<3.4.0,>=3.2.0, but you have bokeh 2.4.3 which is incompatible.
Cloning into 'magenta'...


In [ ]:
!gsutil -q -m cp gs://magentadata/soundfonts/Yamaha-C5-Salamander-JNv5.1.sf2 /content/

# Импортирование нужных библиотек

In [ ]:
import numpy as np

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

from google.colab import files

from tensor2tensor import models
from tensor2tensor import problems
from tensor2tensor.data_generators import text_encoder
from tensor2tensor.utils import decoding
from tensor2tensor.utils import trainer_lib

import note_seq
from magenta.models.score2perf.score2perf import Score2PerfProblem

Instructions for updating:
non-resource variables are not supported in the long term
/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:31: UserWarning: WARN: A Box observation space has an unconventional shape (neither an image, nor a 1D vector). We recommend flattening the observation to have only a 1D vector or use a custom policy to properly process the data. Actual observation shape: (3, 3)
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/cor

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style="darkgrid")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Вспомогательные константы и функции

In [ ]:
SF2_PATH = '/content/Yamaha-C5-Salamander-JNv5.1.sf2'
SAMPLE_RATE = 16000

def decode(ids):
    ids = list(ids)
    if text_encoder.EOS_ID in ids:
        ids = ids[:ids.index(text_encoder.EOS_ID)]
    return encoder.decode(ids)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
def tokens_to_ns(token_ids):
    return note_seq.midi_file_to_note_sequence(decode(token_ids))

In [ ]:
def plot_and_play(ns):
    note_seq.play_sequence(
        ns,
        synth=note_seq.fluidsynth, sample_rate=SAMPLE_RATE, sf2_path=SF2_PATH)
    note_seq.plot_sequence(ns)

In [ ]:
def delete_drums(ns):
    if any(note.is_drum for note in ns.notes):
        notes = [note for note in ns.notes if not note.is_drum]
        del ns.notes[:]
        ns.notes.extend(notes)

In [ ]:
def fix_ns(ns):
    ns = note_seq.apply_sustain_control_changes(ns)
    delete_drums(ns)
    new_notes = sorted([note for note in ns.notes], key=(lambda note: (note.start_time, note.pitch)))
    del ns.notes[:]
    ns.notes.extend(new_notes)
    last_note = {}
    for note in ns.notes:
        note.instrument = 1
        note.program = 0
        if note.pitch in last_note.keys() and last_note[note.pitch].end_time > note.start_time:
            last_note[note.pitch].end_time = note.start_time
        last_note[note.pitch] = note
    return ns

In [ ]:
def ns_prefix(ns, num_notes):
    new_notes = [note for note in ns.notes[:num_notes]]
    del ns.notes[:]
    ns.notes.extend(new_notes)
    ns.total_time = new_notes[-1].end_time

In [ ]:
def decode_to_events(ids):
    ids = list(ids)
    eos_id = text_encoder.EOS_ID
    if eos_id in ids:
        ids = ids[:ids.index(eos_id)]
    performance = note_seq.Performance(
        quantized_sequence=None,
        steps_per_second=100,
        num_velocity_bins=32
    )
    for id in ids:
        performance.append(unconditional_encoders['targets']._encoding.decode_event(id - 2))
    named_events = []
    for event in performance:
        if event.event_type == 1:
            named_event = ('note_on', event.event_value)
        elif event.event_type == 2:
            named_event = ('note_off', event.event_value)
        elif event.event_type == 3:
            named_event = ('time_shift', event.event_value)
        elif event.event_type == 4:
            named_event = ('velocity', 1 + (event.event_value - 1) * 4)
        else:
            named_event = event
        named_events.append(named_event)
    return named_events

# Загрузка модели

In [ ]:
model_name = 'transformer'
hparams_set = 'transformer_tpu'
ckpt_path = 'gs://magentadata/models/music_transformer/checkpoints/unconditional_model_16.ckpt'

class PianoPerformanceLanguageModelProblem(Score2PerfProblem):
    @property
    def add_eos_symbol(self):
        return True

problem = PianoPerformanceLanguageModelProblem()
unconditional_encoders = problem.get_feature_encoders()

hparams = trainer_lib.create_hparams(hparams_set=hparams_set)
trainer_lib.add_problem_hparams(hparams, problem)
hparams.num_hidden_layers = 16
hparams.sampling_method = 'random'

decode_hparams = decoding.decode_hparams()
decode_hparams.alpha = 0.0
decode_hparams.beam_size = 1

run_config = trainer_lib.create_run_config(hparams)
estimator = trainer_lib.create_estimator(
    model_name, hparams, run_config,
    decode_hparams=decode_hparams
)

def input_generator():
  global targets
  global decode_length
  while True:
    yield {
        'targets': np.array([targets], dtype=np.int32),
        'decode_length': np.array(decode_length, dtype=np.int32)
    }

targets = []
decode_length = 0

input_fn = decoding.make_input_fn_from_generator(input_generator())
unconditional_samples = estimator.predict(
    input_fn, checkpoint_path=ckpt_path)

_ = next(unconditional_samples)

Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    
Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
ERROR:tensorfl

In [ ]:
encoder = unconditional_encoders['targets']

In [ ]:
def generate_unconditional(dl=None):
    global targets
    global decode_length
    if dl:
        decode_length = min(dl, 4096 - len(targets))
    else:
        decode_length = max(0, 4096 - len(targets))

    sample_ids = next(unconditional_samples)['outputs']
    return sample_ids

# Определение метрик стилевой разницы

In [ ]:
feature_names = ['note_density', 'pitch_range', 'mean_pitch', 'var_of_pitch',
                'mean_velocity', 'var_of_velocity', 'mean_duration', 'var_of_duration',
                'start_time_mean_diff', 'var_of_start_time_diff']

In [ ]:
def compute_performance_features(ns, start_time, end_time):
    note_list = [note for note in ns.notes if start_time <= note.start_time < end_time]
    note_list = sorted(note_list, key=(lambda note: note.start_time))
    num_notes = len(note_list)
    if num_notes == 0:
        features = [0] * 10
        features[2] = np.mean([note.pitch for note in ns.notes])
        features[6] = 2
        return features
    note_density = num_notes / (end_time - start_time)

    pitch_list = [note.pitch for note in note_list]
    pitch_range = max(pitch_list) - min(pitch_list)

    mean_pitch = np.mean(pitch_list)

    var_of_pitch = np.var(pitch_list)

    velocity_list = [note.velocity for note in note_list]
    mean_velocity = np.mean(velocity_list)

    var_of_velocity = np.var(velocity_list)

    duration_list = [note.end_time - note.start_time for note in note_list]
    mean_duration = np.mean(duration_list)

    var_of_duration = np.var(duration_list)

    start_time_diff = []
    for i in range(1, len(note_list)):
        start_time_diff.append(note_list[i].start_time - note_list[i - 1].start_time)

    if len(start_time_diff) > 0:
        start_time_mean_diff = np.mean(start_time_diff)
        var_of_start_time_diff = np.var(start_time_diff)
    else:
        start_time_mean_diff, var_of_start_time_diff = 0, 0

    features = [note_density, pitch_range, mean_pitch, var_of_pitch,
                mean_velocity, var_of_velocity, mean_duration, var_of_duration,
                start_time_mean_diff, var_of_start_time_diff]
    return features

In [ ]:
def style_difference(ns_1, ns_2, window_size=2, functions={}, epsilons={}, weights=np.ones(10), plot=None):
    if ns_1.total_time < ns_2.total_time:
        max_time = ns_1.total_time
        ns_2 = note_seq.extract_subsequence(ns_2, 0, max_time)
    else:
        max_time = ns_2.total_time
        ns_1 = note_seq.extract_subsequence(ns_1, 0, max_time)
    features_1, features_2 = [], []
    for start_time in np.arange(0, max_time, window_size):
        if start_time + window_size > max_time:
            start_time = max_time - window_size
        features_1.append(compute_performance_features(ns_1, start_time, start_time + window_size))
        features_2.append(compute_performance_features(ns_2, start_time, start_time + window_size))
    features_1 = np.array(features_1)
    features_2 = np.array(features_2)
    error_matrix = np.zeros(features_1.shape)
    for i in range(len(feature_names)):
        f = functions[feature_names[i]]
        eps = epsilons[feature_names[i]]
        error_matrix[:,i] = f(features_1[:,i], features_2[:,i], eps) / weights[i]
    error_matrix[:,[2,4,5]] = 0
    if plot:
        time_axis = np.arange(0, max_time, window_size)
        if plot == 'metric':
            plt.plot(time_axis, error_matrix.mean(1), color='blue')
        elif plot == 'all_features':
            for i, name in enumerate(feature_names):
                plt.plot(time_axis, error_matrix[:, i], label=name)
            plt.legend()
        else:
            idx = feature_names.index(plot)
            plt.plot(time_axis, features_1[:,idx], label='1')
            plt.plot(time_axis, features_2[:,idx], label='2')
            plt.legend()
        plt.show()
    return error_matrix.mean()

In [ ]:
def abs_log_error(f1, f2, eps):
    return np.abs(np.log(f1 + eps) - np.log(f2 + eps))

def abs_error(f1, f2, eps=0):
    return np.abs(f1 - f2)

functions = {name: abs_log_error for name in feature_names}
functions['mean_pitch'] = abs_error

In [ ]:
feature_eps = {name: 1 for name in feature_names}
feature_eps['mean_pitch'] = feature_eps['mean_duration'] = 0
feature_eps['var_of_duration'] = feature_eps['start_time_mean_diff'] = \
        feature_eps['var_of_start_time_diff'] = 0.01

In [ ]:
weights = np.array([0.72521614, 0.69565941, 9.66134072, 1.20305637, 0.42238247,
       1.06960022, 1.05404574, 1.30573771, 0.68013447, 0.71436818])

In [ ]:
def levenshtein(str_1, str_2):
    n, m = len(str_1), len(str_2)
    if n > m:
        str_1, str_2 = str_2, str_1
        n, m = m, n

    current_row = range(n + 1)
    for i in range(1, m + 1):
        previous_row, current_row = current_row, [i] + [0] * n
        for j in range(1, n + 1):
            add, delete, change = previous_row[j] + 1, current_row[j - 1] + 1, previous_row[j - 1]
            if str_1[j - 1] != str_2[i - 1]:
                change += 1
            current_row[j] = min(add, delete, change)

    return current_row[n]

In [ ]:
def get_pitch_list(ns):
    return [note.pitch for note in sorted(ns.notes, key=(lambda n: (n.start_time, n.pitch)))]

In [ ]:
def music_levenshtein(ns_1, ns_2):
    pitches_1 = get_pitch_list(ns_1)
    pitches_2 = get_pitch_list(ns_2)
    min_len = min(len(pitches_1), len(pitches_2))
    pitches_1 = pitches_1[:min_len]
    pitches_2 = pitches_2[:min_len]
    return levenshtein(pitches_1, pitches_2) / min_len

# Основная система

In [ ]:
import os
import random
from tqdm import tqdm

In [ ]:
class GenerativeSystem:
    def upload_track_files(self):
        print('Загрузите файлы с расширением .mid')
        uploaded_files = files.upload()
        self.uploaded_nss = []
        for file_name, file_content in uploaded_files.items():
            if not file_name.endswith('.mid'):
                print(f'Файл "{file_name}" не имеет расширения .mid, он не будет обработан.')
            else:
                ns = note_seq.midi_to_note_sequence(file_content)
                fix_ns(ns)
                self.uploaded_nss.append(ns)

    def upload_track_nss(self, nss):
        self.uploaded_nss = list(nss)

    def generate_similar(self, desired_style_diff=0, continuation_block_1_size=100,
                         continuation_block_2_size=100, block_1_num_samples=100,
                         block_2_num_samples=2, final_num_samples=5):
        global targets
        final_num_samples = min(final_num_samples, block_1_num_samples)
        for file_id, uploaded_ns in enumerate(self.uploaded_nss):
            print(f'Обработка файла номер {file_id + 1}')
            ns_perf_ids = encoder.encode_note_sequence(uploaded_ns)[:-1]
            all_samples_ids = []
            criterion_tuples = []
            for i in tqdm(range(block_1_num_samples)):
                primer_size = random.randint(1, continuation_block_1_size)
                targets = ns_perf_ids[:primer_size]
                sample_ids = generate_unconditional(continuation_block_1_size)
                sample_ids = list(targets) + list(sample_ids)
                all_samples_ids.append(sample_ids)
                ns_generated = tokens_to_ns(sample_ids)
                cur_style_difference = style_difference(
                    uploaded_ns, ns_generated, 3, functions, feature_eps, weights
                )
                cur_levenshtein = music_levenshtein(uploaded_ns, ns_generated)
                criterion_tuple = (cur_levenshtein < 0.5,
                                   abs(cur_style_difference - desired_style_diff),
                                   i)
                criterion_tuples.append(criterion_tuple)
            criterion_tuples.sort()
            similar_samples_ids = []
            for _, _, idx in criterion_tuples[:final_num_samples]:
                similar_samples_ids.append(all_samples_ids[idx])
            for sample_number, cur_sample_ids in enumerate(similar_samples_ids):
                cur_sample_time = tokens_to_ns(cur_sample_ids).total_time
                while cur_sample_ids[-1] != 1 and cur_sample_time < uploaded_ns.total_time:
                    continuation_ids = []
                    sd_error = 1000.0
                    for i in range(block_2_num_samples):
                        targets = cur_sample_ids
                        new_sample_ids = generate_unconditional(continuation_block_2_size)
                        all_sample_ids = list(targets) + list(new_sample_ids)
                        new_ns = tokens_to_ns(all_sample_ids)
                        cur_style_difference = style_difference(
                            uploaded_ns, new_ns, 3, functions, feature_eps, weights
                        )
                        if abs(cur_style_difference - desired_style_diff) < sd_error:
                            sd_error = abs(cur_style_difference - desired_style_diff)
                            continuation_ids = list(new_sample_ids)
                    cur_sample_ids.extend(continuation_ids)
                    cur_sample_time = tokens_to_ns(cur_sample_ids).total_time
                cur_ns = tokens_to_ns(cur_sample_ids)
                plot_and_play(cur_ns)
                file_name = f'sample_{file_id}_{sample_number}.mid'
                note_seq.sequence_proto_to_midi_file(
                    cur_ns, file_name
                )
                print(f'Сэмпл сохранён в файл "{file_name}"')

In [ ]:
gen_system = GenerativeSystem()

In [ ]:
gen_system.upload_track_files()

Загрузите файлы с расширением .mid


Saving fur_elise.mid to fur_elise (4).mid


In [ ]:
gen_system.generate_similar()

Обработка файла номер 1


100%|██████████| 100/100 [09:36<00:00,  5.76s/it]


Сэмпл сохранён в файл "sample_0_0.mid"


Сэмпл сохранён в файл "sample_0_1.mid"


Сэмпл сохранён в файл "sample_0_2.mid"


Сэмпл сохранён в файл "sample_0_3.mid"


Сэмпл сохранён в файл "sample_0_4.mid"
